In [1]:
library(tidyverse)
library(here)
library(glue)
library(furrr)
suppressPackageStartupMessages(library(rtracklayer))
devtools::load_all(".")

plan(multiprocess, workers = 20)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
here() starts at /mnt/expressions/mp/archaic-ychr

Attaching package: ‘glue’

The following object is masked from ‘package:dplyr’:

    collapse

Loading required package: future
Loading ychr


# Full 6.9 Mb capture

In [2]:
samples_full <- list.files(here("data/vcf"), "full_.*.vcf.gz$") %>%
    str_subset("mez2_dp|modern|merged", negate = T) %>%
    str_replace("full_(.*).vcf.gz", "\\1")

In [3]:
samples_full

[1] "a00_1"             "a00_2"             "a00"              
 [4] "chimp"             "den_snpad"         "den"              
 [7] "den4"              "den8"              "mez2_snpad"       
[10] "mez2"              "S_BedouinB-1"      "S_Burmese-1"      
[13] "S_Dai-2"           "S_Dinka-1"         "S_Finnish-2"      
[16] "S_French-1"        "S_Gambian-1"       "S_Han-2"          
[19] "S_Ju_hoan_North-1" "S_Karitiana-1"     "S_Mandenka-1"     
[22] "S_Mbuti-1"         "S_Papuan-2"        "S_Punjabi-1"      
[25] "S_Saami-2"         "S_Sardinian-1"     "S_Thai-1"         
[28] "S_Turkish-1"       "S_Yoruba-2"        "shotgun_mez2"     
[31] "shotgun_spy1"      "spy1"              "ustishim"

In [4]:
bed_full <- here("data/coord/capture_full.bed")

In [5]:
capture_full <- import.bed(bed_full)

In [6]:
sum(width(capture_full))

[1] 6913053

In [7]:
full_counts <- future_map_dfr(samples_full, function(ind) {
    gt_all <- read_vcf(here(glue("data/vcf/full_{ind}.vcf.gz")), mindp = 3, maxdp = 0.98, bed_filter = bed_full)
    gt_nodmg <- read_vcf(here(glue("data/vcf/full_{ind}.vcf.gz")), mindp = 3, maxdp = 0.98, nodmg = T, bed_filter = bed_full)

    tibble(
        individual = ind,
        total = sum(width(capture_full)),
        all = nrow(gt_all),
        all_prop = all / total,
        nodmg = nrow(gt_nodmg),
        nodmg_prop = nodmg / total
    )
})

In [8]:
full_counts %>%
    arrange(all) %>%
    mutate_at(vars(contains("prop")), ~ sprintf("%.2f%%", 100 * .)) %>%
    select(-total) %>%
    filter(!individual %in% c("den", "den_snpad", "a00_1", "a00_2", "chimp")) %>%
    mutate(name = fix_name(individual)) %>%
    select(name, everything(), -individual)

name,all,all_prop,nodmg,nodmg_prop
<chr>,<int>,<chr>,<int>,<chr>
Spy 94a (shotgun),2664973,38.55%,2636589,38.14%
Spy 94a,3502527,50.67%,3470817,50.21%
Mezmaiskaya 2 (shotgun),3822247,55.29%,3762906,54.43%
Denisova 4,4731419,68.44%,4705977,68.07%
Denisova 8,5851442,84.64%,5828466,84.31%
Mezmaiskaya 2,6349143,91.84%,6346879,91.81%
Mezmaiskaya 2 (snpAD),6670132,96.49%,6667661,96.45%
A00,6873362,99.43%,6872468,99.41%
Ust'-Ishim,6888333,99.64%,6888248,99.64%


# 560 kb capture of regions from Lippold et al.

In [9]:
samples_lippold <- list.files(here("data/vcf"), "lippold_.*.vcf.gz$") %>%
    str_subset("mez2_dp|modern|merged", negate = T) %>%
    str_replace("lippold_(.*).vcf.gz", "\\1")

In [10]:
samples_lippold

[1] "a00_1"             "a00_2"             "a00"              
 [4] "chimp"             "den4"              "den8"             
 [7] "elsidron2"         "mez2"              "S_BedouinB-1"     
[10] "S_Burmese-1"       "S_Dai-2"           "S_Dinka-1"        
[13] "S_Finnish-2"       "S_French-1"        "S_Gambian-1"      
[16] "S_Han-2"           "S_Ju_hoan_North-1" "S_Karitiana-1"    
[19] "S_Mandenka-1"      "S_Mbuti-1"         "S_Papuan-2"       
[22] "S_Punjabi-1"       "S_Saami-2"         "S_Sardinian-1"    
[25] "S_Thai-1"          "S_Turkish-1"       "S_Yoruba-2"       
[28] "spy1"              "ustishim"

In [11]:
bed_lippold <- here("data/coord/capture_lippold.bed")

In [12]:
capture_lippold <- import.bed(bed_lippold)

In [13]:
sum(width(capture_lippold))

[1] 556259

In [14]:
counts_lippold <- future_map_dfr(samples_lippold, function(ind) {
    gt_all <- read_vcf(here(glue("data/vcf/lippold_{ind}.vcf.gz")), mindp = 3, maxdp = 0.98, bed_filter = bed_lippold)
    gt_nodmg <- read_vcf(here(glue("data/vcf/lippold_{ind}.vcf.gz")), mindp = 3, maxdp = 0.98, nodmg = T, bed_filter = bed_lippold)

    tibble(
        individual = ind,
        total = sum(width(capture_lippold)),
        all = nrow(gt_all),
        all_prop = all / total,
        nodmg = nrow(gt_nodmg),
        nodmg_prop = nodmg / total
    )
})

In [15]:
counts_lippold %>%
    arrange(all) %>%
    mutate_at(vars(contains("prop")), ~ sprintf("%.2f%%", 100 * .)) %>%
    select(-total) %>%
    filter(!individual %in% c("den", "den_snpad", "mez2_snpad", "a00_1", "a00_2", "chimp")) %>%
    mutate(name = fix_name(individual)) %>%
    select(name, everything(), -individual)

name,all,all_prop,nodmg,nodmg_prop
<chr>,<int>,<chr>,<int>,<chr>
Spy 94a,275626,49.55%,272798,49.04%
Denisova 4,408775,73.49%,406508,73.08%
Denisova 8,484377,87.08%,482608,86.76%
Mezmaiskaya 2,509145,91.53%,508962,91.50%
El Sidrón 1253 (560 kb),530172,95.31%,530046,95.29%
S_Yoruba-2,553203,99.45%,553112,99.43%
S_Mandenka-1,553348,99.48%,553257,99.46%
S_Gambian-1,553365,99.48%,553275,99.46%
S_Mbuti-1,553381,99.48%,553288,99.47%
